# Week 8: Data Pipeline & CRUD (SQL)
## CST1510 — Multi-Domain Intelligence Platform

**Building Your Database Layer**

<hr style="border:2px solid #0EA5E9">

### What You'll Build This Week

This week, you're transitioning from **file-based storage** (`users.txt`) to a **professional database system** (SQLite). By the end of this lab, you will have:

1.  **Migrated** your Week 7 users from `users.txt` → SQLite database
2.  **Created** database tables for all three domains (cyber_incidents, datasets_metadata, it_tickets)
3.  **Loaded** CSV data using pandas
4.  **Implemented** CRUD operations (Create, Read, Update, Delete) using Python functions
5.  **Secured** your queries against SQL injection attacks
6.  **Tested** your database with real-world queries

### Learning Objectives

By completing this lab, you will:

- Understand **why databases are better** than text files for data storage
- Learn how to **connect to SQLite** using Python's built-in `sqlite3` module
- Write **SQL CREATE TABLE** statements to define your data structure
- Implement **CRUD operations** using Python functions
- Use **parameterized queries** to prevent SQL injection
- Load **CSV files** efficiently using pandas
- Query your database to extract **meaningful insights**

###  Beginner Tip

Think of a database like a **super-powered Excel file** that:
- Lives on disk (persists data)
- Lets you search, add, update, and delete data **without reading the whole file**
- Can link related data together (users → incidents)
- Protects against data corruption

---

## Part 0: Prerequisites & Setup

### Step 0.1: Check Your Project Structure

Before starting, make sure your project follows this structure:

```
CW2_M0123456_CST1510/
│
├─ app/
│  └─ data/              # Your database functions will go here
│
├─ DATA/                 # IMPORTANT: Uppercase DATA folder
│  ├─ users.txt          # From Week 7
│  ├─ cyber_incidents.csv
│  ├─ datasets_metadata.csv
│  ├─ it_tickets.csv
│  └─ intelligence_platform.db  # Will be created by your code
│
└─ requirements.txt
```

### Step 0.2: Install Required Libraries

We'll use:
- `sqlite3` → **Built-in** to Python (no install needed!)
- `pandas` → For easy CSV loading
- `bcrypt` → For password hashing (from Week 7)

Run this cell to install the required packages:

In [88]:
# Install required packages
%pip install pandas bcrypt

Note: you may need to restart the kernel to use updated packages.


### Step 0.3: Import Modules and Define Constants

Let's import everything we need and set up our paths:

In [89]:
import sqlite3
import pandas as pd
import bcrypt
from pathlib import Path

# Define paths
DATA_DIR = Path("DATA")
DB_PATH = DATA_DIR / "intelligence_platform.db"

# Create DATA folder if it doesn't exist
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(" Imports successful!")
print(f" DATA folder: {DATA_DIR.resolve()}")
print(f" Database will be created at: {DB_PATH.resolve()}")

 Imports successful!
 DATA folder: C:\Users\vingp\Downloads\week 8 lab\week 8 lab\DATA
 Database will be created at: C:\Users\vingp\Downloads\week 8 lab\week 8 lab\DATA\intelligence_platform.db


---

## Part 1: Why Move from Files to Databases?

### Understanding the Problem

In Week 7, you stored users in `users.txt`. This works for small projects, but has serious limitations:

| **File Storage** (`users.txt`) | **Database** (`intelligence_platform.db`) |
|--------------------------------|-------------------------------------------|
| Slow search (must read entire file) | ⚡ Fast search with SQL queries |
| No relationships between data |  Link users to incidents, tickets, etc. |
| Risk of corruption | ACID-safe (Atomicity, Consistency, Isolation, Durability) |
| Manual parsing required | Powerful query language (SQL) |
| Single-user access | Multi-user support |

### Your Database Schema

You'll create **4 tables**:

1. **`users`** — User accounts with authentication
2. **`cyber_incidents`** — Security incidents (your chosen domain)
3. **`datasets_metadata`** — Dataset information
4. **`it_tickets`** — IT support tickets

---

## Part 2: Database Connection Functions

### Step 2.1: Create Connection Function

First, we need a function to connect to our database. This function will:
- Create the database file if it doesn't exist
- Return a connection object that we can use to run SQL commands

 **Beginner Tip**: Think of `conn` (connection) as a phone line to your database. You need it to send commands and get responses.

In [90]:
def connect_database(db_path=DB_PATH):
    """
    Connect to the SQLite database.
    Creates the database file if it doesn't exist.
    
    Args:
        db_path: Path to the database file
        
    Returns:
        sqlite3.Connection: Database connection object
    """
    return sqlite3.connect(str(db_path))

---

## Part 3: Create Database Tables

### Step 3.1: Create the `users` Table

Let's start by creating a table for users. This table will store:
- `id` — Unique identifier (auto-incremented)
- `username` — User's login name (must be unique)
- `password_hash` — Hashed password (from bcrypt)
- `role` — User role (e.g., 'user', 'analyst', 'admin')

💡 **Beginner Tip**: `CREATE TABLE IF NOT EXISTS` means "create this table only if it doesn't already exist". This prevents errors if you run the code multiple times.

In [91]:
def create_users_table(conn):
    """
    Create the users table if it doesn't exist.
    
    This is a COMPLETE IMPLEMENTATION as an example.
    Study this carefully before implementing the other tables!
    
    Args:
        conn: Database connection object
    """
    cursor = conn.cursor()
    
    # SQL statement to create users table
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT NOT NULL UNIQUE,
        password_hash TEXT NOT NULL,
        role TEXT DEFAULT 'user',
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    
    cursor.execute(create_table_sql)
    conn.commit()
    print("✅ Users table created successfully!")

### Step 3.2: Create Domain Tables

Now let's create tables for your three domains. Each table will have columns matching your CSV files.

#### Security Note: Foreign Keys

Notice that `cyber_incidents` has a `FOREIGN KEY` that references `users(username)`. This creates a **relationship** between tables:
- Each incident can be linked to the user who reported it
- This is one of the key advantages of databases over text files!

In [92]:
def create_cyber_incidents_table(conn):
    """
    Create the cyber_incidents table.
    """
    cursor = conn.cursor()

    sql = """
    CREATE TABLE IF NOT EXISTS cyber_incidents (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT,
        incident_type TEXT,
        severity TEXT,
        status TEXT,
        description TEXT,
        reported_by TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (reported_by) REFERENCES users(username)
    )
    """

    cursor.execute(sql)
    conn.commit()
    print("✅ cyber_incidents table created successfully!")
    pass


def create_datasets_metadata_table(conn):
    """
    Create the datasets_metadata table.
    """
    cursor = conn.cursor()

    sql = """
    CREATE TABLE IF NOT EXISTS datasets_metadata (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        dataset_name TEXT NOT NULL,
        category TEXT,
        source TEXT,
        last_updated TEXT,
        record_count INTEGER,
        file_size_mb REAL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """

    cursor.execute(sql)
    conn.commit()
    print("✅ datasets_metadata table created successfully!")


def create_it_tickets_table(conn):
    """
    Create the it_tickets table.
    """
    cursor = conn.cursor()

    sql = """
    CREATE TABLE IF NOT EXISTS it_tickets (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ticket_id TEXT NOT NULL,
        priority TEXT,
        status TEXT,
        category TEXT,
        subject TEXT NOT NULL,
        description TEXT,
        created_date TEXT,
        resolved_date TEXT,
        assigned_to TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """

    cursor.execute(sql)
    conn.commit()
    print("✅ it_tickets table created successfully!")
    pass

---

## Part 4: Migrate Users from Week 7

### Step 4.1: Understanding Migration

**Migration** means copying data from an old format (text file) to a new format (database table).

Your `users.txt` file from Week 7 has this format:
```
username,password_hash,role
alice,$2b$12$...,analyst
bob,$2b$12$...,user
```

We need to:
1. Read each line from `users.txt`
2. Parse the username, password_hash, and role
3. INSERT each user into the `users` table

### Step 4.2: Create Migration Function

In [93]:
def migrate_users_from_file(conn, filepath=DATA_DIR / "users.txt"):
    """
    Migrate users from users.txt to the database.
    
    This is a COMPLETE IMPLEMENTATION as an example.
    
    Args:
        conn: Database connection
        filepath: Path to users.txt file
    """
    if not filepath.exists():
        print(f"⚠️  File not found: {filepath}")
        print("   No users to migrate.")
        return
    
    cursor = conn.cursor()
    migrated_count = 0
    
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            # Parse line: username,password_hash
            parts = line.split(',')
            if len(parts) >= 2:
                username = parts[0]
                password_hash = parts[1]
                
                # Insert user (ignore if already exists)
                try:
                    cursor.execute(
                        "INSERT OR IGNORE INTO users (username, password_hash, role) VALUES (?, ?, ?)",
                        (username, password_hash, 'user')
                    )
                    if cursor.rowcount > 0:
                        migrated_count += 1
                except sqlite3.Error as e:
                    print(f"Error migrating user {username}: {e}")
    
    conn.commit()
    print(f"✅ Migrated {migrated_count} users from {filepath.name}")

### Step 4.3: Verify Migration

Let's check that the users were actually inserted into the database:

In [94]:
# Verify users were migrated
conn = connect_database()
cursor = conn.cursor()

# Query all users
cursor.execute("SELECT id, username, role FROM users")
users = cursor.fetchall()

print(" Users in database:")
print(f"{'ID':<5} {'Username':<15} {'Role':<10}")
print("-" * 35)
for user in users:
    print(f"{user[0]:<5} {user[1]:<15} {user[2]:<10}")

print(f"\nTotal users: {len(users)}")
conn.close()

 Users in database:
ID    Username        Role      
-----------------------------------
1     manan           user      
2     test_user       user      
10    suriya          user      

Total users: 3


---

## Part 5: Authentication Functions (Database-Backed)

### Step 5.1: Register Function

Now that users are in the database, we need to update our authentication to work with the database instead of `users.txt`.

The `register()` function will:
1. Check if username already exists
2. Hash the password with bcrypt
3. INSERT the new user into the database

In [95]:
def register_user(username, password, role="user"):
    """
    Register a new user in the database.
    
    This is a COMPLETE IMPLEMENTATION as an example.
    
    Args:
        username: User's login name
        password: Plain text password (will be hashed)
        role: User role (default: 'user')
        
    Returns:
        tuple: (success: bool, message: str)
    """
    conn = connect_database()
    cursor = conn.cursor()
    
    # Check if user already exists
    cursor.execute("SELECT * FROM users WHERE username = ?", (username,))
    if cursor.fetchone():
        conn.close()
        return False, f"Username '{username}' already exists."
    
    # Hash the password
    password_bytes = password.encode('utf-8')
    salt = bcrypt.gensalt()
    hashed = bcrypt.hashpw(password_bytes, salt)
    password_hash = hashed.decode('utf-8')
    
    # Insert new user
    cursor.execute(
        "INSERT INTO users (username, password_hash, role) VALUES (?, ?, ?)",
        (username, password_hash, role)
    )
    conn.commit()
    conn.close()
    
    return True, f"User '{username}' registered successfully!"

### Step 5.2: Login Function

The `login()` function will:
1. Look up the user in the database
2. Retrieve their stored password hash
3. Verify the provided password against the hash using bcrypt

In [96]:
def login_user(username, password):
    """
    Authenticate a user against the database.
    
    This is a COMPLETE IMPLEMENTATION as an example.
    
    Args:
        username: User's login name
        password: Plain text password to verify
        
    Returns:
        tuple: (success: bool, message: str)
    """
    conn = connect_database()
    cursor = conn.cursor()
    
    # Find user
    cursor.execute("SELECT * FROM users WHERE username = ?", (username,))
    user = cursor.fetchone()
    conn.close()
    
    if not user:
        return False, "Username not found."
    
    # Verify password (user[2] is password_hash column)
    stored_hash = user[2]
    password_bytes = password.encode('utf-8')
    hash_bytes = stored_hash.encode('utf-8')
    
    if bcrypt.checkpw(password_bytes, hash_bytes):
        return True, f"Welcome, {username}!"
    else:
        return False, "Invalid password."

---

## Part 6: Load CSV Data with Pandas

### Step 6.1: Understanding Bulk Loading

Now that your tables exist, you can load the provided CSV files. Pandas makes this incredibly easy with the `to_sql()` method.

 **Beginner Tip**: 
- `if_exists='append'` means "add to existing data"
- `if_exists='replace'` means "delete old data and insert new"
- `index=False` means "don't save the DataFrame index as a column"

### Step 6.2: Create CSV Loading Function

In [97]:
def load_csv_to_table(conn, csv_path, table_name):
    """
    Load a CSV file into a database table using pandas.
    Maps CSV columns to table schema for each domain.
    """
    csv_path = Path(csv_path)

    if not csv_path.exists():
        print(f"CSV not found: {csv_path}")
        return 0

    df = pd.read_csv(csv_path)

    # Map CSV columns to table schema based on table_name
    if table_name == "cyber_incidents":
        # CSV: Title, Date, Type, Category → Table: description, date, incident_type, severity
        df = df.rename(columns={
            'Title': 'description',
            'Date': 'date',
            'Type': 'incident_type',
            'Category': 'severity'
        })
        # Select only columns that exist in the table
        df = df[['date', 'incident_type', 'severity', 'description']]
        df['status'] = 'Open'  # Default status
        df['reported_by'] = None  # No reporter info in CSV
    
    elif table_name == "datasets_metadata":
        # CSV: name, source, category, size_mb → Table: dataset_name, source, category, file_size_mb
        df = df.rename(columns={
            'name': 'dataset_name',
            'size_mb': 'file_size_mb'
        })
        # Select only columns that exist in the table
        df = df[['dataset_name', 'category', 'source', 'file_size_mb']]
        df['last_updated'] = None  # No date info in CSV
        df['record_count'] = None  # No record count in CSV
    
    elif table_name == "it_tickets":
        # CSV already has matching columns, just drop 'id' (will be auto-generated)
        df = df.drop(columns=['id'], errors='ignore')
        # Ensure all required columns exist
        required_cols = ['ticket_id', 'priority', 'status', 'category', 'subject', 
                        'description', 'created_date', 'resolved_date', 'assigned_to']
        df = df[required_cols]

    df.to_sql(
        name=table_name,
        con=conn,
        if_exists='append',
        index=False
    )

    print(f"✅ Loaded {len(df)} rows into '{table_name}'")

    return len(df)

---

## Part 7: CRUD Operations

### Understanding CRUD

**CRUD** stands for the four basic operations you can perform on database data:

| Operation | SQL Command | What It Does |
|-----------|-------------|-------------|
| **C**reate | `INSERT` | Add new records |
| **R**ead | `SELECT` | Retrieve existing records |
| **U**pdate | `UPDATE` | Modify existing records |
| **D**elete | `DELETE` | Remove records |

###  Security: Parameterized Queries

**CRITICAL**: Always use `?` placeholders and pass values as a tuple to prevent SQL injection attacks!

 **NEVER DO THIS** (vulnerable to SQL injection):
```python
query = f"SELECT * FROM users WHERE username = '{username}'"
```

 **ALWAYS DO THIS** (safe):
```python
query = "SELECT * FROM users WHERE username = ?"
cursor.execute(query, (username,))
```

---

### Step 7.1: CREATE — Insert New Incident

In [98]:
def insert_incident(conn, date, incident_type, severity, status, description, reported_by=None):
    """
    Insert a new incident into the database.
    """
    cursor = conn.cursor()

    sql = """
    INSERT INTO cyber_incidents 
    (date, incident_type, severity, status, description, reported_by)
    VALUES (?, ?, ?, ?, ?, ?)
    """

    cursor.execute(sql, (date, incident_type, severity, status, description, reported_by))
    conn.commit()

    return cursor.lastrowid
    pass

### Step 7.2: READ — Query Incidents

In [99]:
def get_all_incidents(conn):
    """
    Return all cyber incidents as a pandas DataFrame.
    """
    query = "SELECT * FROM cyber_incidents ORDER BY id DESC"
    df = pd.read_sql_query(query, conn)
    return df
    pass

### Step 7.3: UPDATE — Modify Incident Status

In [100]:
def update_incident_status(conn, incident_id, new_status):
    """
    Update the status of an incident.
    """
    cursor = conn.cursor()

    sql = """
    UPDATE cyber_incidents
    SET status = ?
    WHERE id = ?
    """

    cursor.execute(sql, (new_status, incident_id))
    conn.commit()

    return cursor.rowcount
    pass

### Step 7.4: DELETE — Remove Incident

 **WARNING**: DELETE is permanent! Always use a WHERE clause to avoid deleting all rows.

In [101]:
def delete_incident(conn, incident_id):
    """
    Delete an incident by ID.
    """
    cursor = conn.cursor()

    sql = "DELETE FROM cyber_incidents WHERE id = ?"

    cursor.execute(sql, (incident_id,))
    conn.commit()

    return cursor.rowcount
    pass

---

## Part 8: Analytical Queries (The Big 6) - OPTIONAL it couuld be done with pandas

### Step 8.1: Using GROUP BY for Aggregation

Let's use the **Big 6 SQL clauses** to extract insights from your data:

1. **SELECT** — Choose what columns to return
2. **FROM** — Specify the table
3. **WHERE** — Filter individual rows
4. **GROUP BY** — Group rows for aggregation
5. **HAVING** — Filter aggregated groups
6. **ORDER BY** — Sort the results

In [102]:
def get_incidents_by_type_count(conn):
    """
    Count incidents by type.
    Uses: SELECT, FROM, GROUP BY, ORDER BY
    """
    query = """
    SELECT incident_type, COUNT(*) as count
    FROM cyber_incidents
    GROUP BY incident_type
    ORDER BY count DESC
    """
    df = pd.read_sql_query(query, conn)
    return df

def get_high_severity_by_status(conn):
    """
    Count high severity incidents by status.
    Uses: SELECT, FROM, WHERE, GROUP BY, ORDER BY
    """
    query = """
    SELECT status, COUNT(*) as count
    FROM cyber_incidents
    WHERE severity = 'High'
    GROUP BY status
    ORDER BY count DESC
    """
    df = pd.read_sql_query(query, conn)
    return df

def get_incident_types_with_many_cases(conn, min_count=5):
    """
    Find incident types with more than min_count cases.
    Uses: SELECT, FROM, GROUP BY, HAVING, ORDER BY
    """
    query = """
    SELECT incident_type, COUNT(*) as count
    FROM cyber_incidents
    GROUP BY incident_type
    HAVING COUNT(*) > ?
    ORDER BY count DESC
    """
    df = pd.read_sql_query(query, conn, params=(min_count,))
    return df

# Test: Run analytical queries
conn = connect_database()

print("\n Incidents by Type:")
df_by_type = get_incidents_by_type_count(conn)
print(df_by_type)

print("\n High Severity Incidents by Status:")
df_high_severity = get_high_severity_by_status(conn)
print(df_high_severity)

print("\n Incident Types with Many Cases (>5):")
df_many_cases = get_incident_types_with_many_cases(conn, min_count=5)
print(df_many_cases)

conn.close()


 Incidents by Type:
       incident_type  count
0          Espionage   3366
1               None    306
2           Sabotage    207
3  Denial of service    162
4   Data destruction    126
5    Financial Theft     63
6             Doxing     54
7         Defacement     45

 High Severity Incidents by Status:
Empty DataFrame
Columns: [status, count]
Index: []

 Incident Types with Many Cases (>5):
       incident_type  count
0          Espionage   3366
1               None    306
2           Sabotage    207
3  Denial of service    162
4   Data destruction    126
5    Financial Theft     63
6             Doxing     54
7         Defacement     45


---

## Part 9: Complete Database Setup Script

### Step 9.1: Create a Complete Setup Function

Let's create a single function that sets up your entire database from scratch:

In [103]:
def setup_database_complete():
    """
    Complete database setup:
    1. Connect to database
    2. Create all tables
    3. Migrate users from users.txt
    4. Load CSV data for all domains
    5. Verify setup
    """
    print("\n" + "="*60)
    print("STARTING COMPLETE DATABASE SETUP")
    print("="*60)
    
    # Step 1: Connect
    print("\n[1/5] Connecting to database...")
    conn = connect_database()
    print("✅ Connected")
    
    # Step 2: Create tables
    print("\n[2/5] Creating database tables...")
    create_users_table(conn)
    create_cyber_incidents_table(conn)
    create_datasets_metadata_table(conn)
    create_it_tickets_table(conn)
    
    # Step 3: Migrate users
    print("\n[3/5] Migrating users from users.txt...")
    migrate_users_from_file(conn)
    
    # Step 4: Load CSV data
    print("\n[4/5] Loading CSV data...")
    load_csv_to_table(conn, DATA_DIR / "cyber_incidents.csv", "cyber_incidents")
    load_csv_to_table(conn, DATA_DIR / "datasets_metadata.csv", "datasets_metadata")
    load_csv_to_table(conn, DATA_DIR / "it_tickets.csv", "it_tickets")
    
    # Step 5: Verify
    print("\n[5/5] Verifying database setup...")
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables_found = [r[0] for r in cursor.fetchall()]
    print("✅ Tables in DB:", tables_found)
    
    # Count rows in each table
    tables = ['users', 'cyber_incidents', 'datasets_metadata', 'it_tickets']
    print("\n📊 Database Summary:")
    print(f"{'Table':<25} {'Row Count':<15}")
    print("-" * 40)
    
    for table in tables:
        try:
            cursor.execute(f"SELECT COUNT(*) FROM {table}")
            count = cursor.fetchone()[0]
            print(f"{table:<25} {count:<15}")
        except sqlite3.OperationalError:
            print(f"{table:<25} {'N/A (not found)':<15}")
    
    conn.close()
    
    print("\n" + "="*60)
    print("✅ DATABASE SETUP COMPLETE!")
    print("="*60)
    print(f"\n📁 Database location: {DB_PATH.resolve()}")
    print("🎉 You're ready for Week 9 (Streamlit web interface)!")

# Run the complete setup
setup_database_complete()


STARTING COMPLETE DATABASE SETUP

[1/5] Connecting to database...
✅ Connected

[2/5] Creating database tables...
✅ Users table created successfully!
✅ cyber_incidents table created successfully!
✅ datasets_metadata table created successfully!
✅ it_tickets table created successfully!

[3/5] Migrating users from users.txt...
⚠️  File not found: DATA\users.txt
   No users to migrate.

[4/5] Loading CSV data...
✅ Loaded 481 rows into 'cyber_incidents'
✅ Loaded 1000 rows into 'datasets_metadata'
✅ Loaded 1000 rows into 'it_tickets'

[5/5] Verifying database setup...
✅ Tables in DB: ['users', 'sqlite_sequence', 'cyber_incidents', 'datasets_metadata', 'it_tickets']

📊 Database Summary:
Table                     Row Count      
----------------------------------------
users                     3              
cyber_incidents           4810           
datasets_metadata         10000          
it_tickets                10000          

✅ DATABASE SETUP COMPLETE!

📁 Database location: C:\Users\v

---

## Part 10: Testing & Verification

### Step 10.1: Comprehensive Database Test

In [104]:
def run_comprehensive_tests():
    """
    Run comprehensive tests on your database.
    """
    print("\n" + "="*60)
    print("🧪 RUNNING COMPREHENSIVE TESTS")
    print("="*60)
    
    conn = connect_database()
    
    # Test 1: Authentication
    print("\n[TEST 1] Authentication")
    success, msg = register_user("test_user", "TestPass123!", "user")
    print(f"  Register: {'✅' if success else '❌'} {msg}")
    
    success, msg = login_user("test_user", "TestPass123!")
    print(f"  Login:    {'✅' if success else '❌'} {msg}")
    
    # Test 2: CRUD Operations
    print("\n[TEST 2] CRUD Operations")
    
    # Create
    test_id = insert_incident(
        conn,
        "2024-11-05",
        "Test Incident",
        "Low",
        "Open",
        "This is a test incident",
        "test_user"
    )
    print(f"  Create: ✅ Incident #{test_id} created")
    
    # Read
    df = pd.read_sql_query(
        "SELECT * FROM cyber_incidents WHERE id = ?",
        conn,
        params=(test_id,)
    )
    print(f"  Read:    Found incident #{test_id}")
    
    # Update
    update_incident_status(conn, test_id, "Resolved")
    print(f"  Update:  Status updated")
    
    # Delete
    delete_incident(conn, test_id)
    print(f"  Delete:  Incident deleted")
    
    # Test 3: Analytical Queries
    print("\n[TEST 3] Analytical Queries")
    
    df_by_type = get_incidents_by_type_count(conn)
    print(f"  By Type:     Found {len(df_by_type)} incident types")
    
    df_high = get_high_severity_by_status(conn)
    print(f"  High Severity: Found {len(df_high)} status categories")
    
    conn.close()
    
    print("\n" + "="*60)
    print("✅ ALL TESTS PASSED!")
    print("="*60)

# Run tests
run_comprehensive_tests()


🧪 RUNNING COMPREHENSIVE TESTS

[TEST 1] Authentication
  Register: ❌ Username 'test_user' already exists.
  Login:    ✅ Welcome, test_user!

[TEST 2] CRUD Operations
  Create: ✅ Incident #4820 created
  Read:    Found incident #4820
  Update:  Status updated
  Delete:  Incident deleted

[TEST 3] Analytical Queries
  By Type:     Found 8 incident types
  High Severity: Found 0 status categories

✅ ALL TESTS PASSED!


---

##  Congratulations!

### What You've Accomplished

You've successfully:

 **Migrated** from file-based storage to a professional SQLite database  
 **Created** a complete database schema with 4 tables  
 **Implemented** secure authentication with bcrypt  
 **Loaded** CSV data efficiently using pandas  
 **Built** CRUD functions for all database operations  
 **Secured** your queries against SQL injection  
 **Extracted** insights using analytical SQL queries  

### Your Database Structure

```
intelligence_platform.db
├─ users                 (authentication)
├─ cyber_incidents       (security domain)
├─ datasets_metadata     (data domain)
└─ it_tickets            (IT domain)
```

---

##  Next Steps: Week 9 Preview

### What's Coming in Week 9

Next week, you'll build a **Streamlit web interface** that uses your database:

1. **Login Page** — Use your `login_user()` function
2. **Dashboard** — Display incident statistics with charts
3. **CRUD Forms** — Interactive forms for creating/updating incidents
4. **Visualizations** — Use Plotly to create interactive charts
5. **Session Management** — Keep users logged in across pages

### Preparing for Week 9

Make sure your database is working correctly:
-  All tables created
-  Data loaded from CSVs
-  CRUD operations tested
-  Queries returning correct results

---

##  Submission Checklist

Before submitting your Week 8 work, ensure you have:

### Files to Submit

- [ ] `app/data/db.py` — Database connection functions
- [ ] `app/data/schema.py` — CREATE TABLE statements
- [ ] `app/data/users.py` — User CRUD functions
- [ ] `app/data/incidents.py` — Incident CRUD functions
- [ ] `app/data/datasets.py` — Dataset CRUD functions
- [ ] `app/data/tickets.py` — Ticket CRUD functions
- [ ] `app/services/user_service.py` — User migration function
- [ ] `main.py` — Demo script showing all CRUD operations
- [ ] `DATA/intelligence_platform.db` — Your populated database
- [ ] `requirements.txt` — Updated with pandas, bcrypt
- [ ] `docs/README.md` — Documentation with screenshots

### Testing Checklist

- [ ] Database connects successfully
- [ ] All 4 tables created
- [ ] Users migrated from users.txt
- [ ] CSV data loaded
- [ ] Registration works
- [ ] Login works
- [ ] Can create new incidents
- [ ] Can read/query incidents
- [ ] Can update incident status
- [ ] Can delete incidents
- [ ] Analytical queries return results
- [ ] No SQL injection vulnerabilities (all queries use `?` placeholders)

---

##  Tips & Best Practices

### Database Best Practices

1. **Always close connections** when done
2. **Use parameterized queries** (never string formatting)
3. **Commit after writes** (INSERT, UPDATE, DELETE)
4. **Use transactions** for multiple related operations
5. **Index frequently queried columns** (for performance)

### Debugging Tips

If something doesn't work:

1. **Check the error message** — SQL errors are usually descriptive
2. **Print your SQL** — Use `print(query)` to see what's being executed
3. **Test queries in DB Browser** — Use a GUI tool to test SQL
4. **Check data types** — Make sure your Python types match SQL types
5. **Verify file paths** — Use absolute paths or check current directory

### Common Errors

| Error | Cause | Solution |
|-------|-------|----------|
| `table already exists` | Running CREATE TABLE twice | Use `IF NOT EXISTS` |
| `UNIQUE constraint failed` | Duplicate username/ID | Check before INSERT |
| `no such table` | Table not created | Run CREATE TABLE first |
| `no such column` | Typo in column name | Check table schema |
| `database is locked` | Connection not closed | Always close connections |

---

##  Additional Resources

### SQLite Documentation
- [SQLite Official Docs](https://www.sqlite.org/docs.html)
- [Python sqlite3 Module](https://docs.python.org/3/library/sqlite3.html)

### SQL Learning Resources
- [W3Schools SQL Tutorial](https://www.w3schools.com/sql/)
- [SQLite Tutorial](https://www.sqlitetutorial.net/)

### Tools
- [DB Browser for SQLite](https://sqlitebrowser.org/) — GUI for viewing/editing databases
- [SQLite Viewer (VS Code Extension)](https://marketplace.visualstudio.com/items?itemName=alexcvzz.vscode-sqlite)

---
